<a href="https://colab.research.google.com/github/mprksa/blocks/blob/main/training2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Bagian 1: Setup dan Instalasi**

In [ ]:
# Import Dataset from GitHub
!git clone https://github.com/mprksa/Blocks2.git

In [ ]:
# Cek versi Python dan install paket yang diperlukan
!python --version
!pip install --upgrade pip
!pip install mediapipe-model-maker

# **Bagian 2: Import dan Load Data**

In [ ]:
# Import pustaka yang diperlukan
import os
import json
import tensorflow as tf
assert tf.__version__.startswith('2')
from mediapipe_model_maker import object_detector

In [ ]:
# Path ke dataset
train_dataset_path = "blocks/train"
validation_dataset_path = "blocks/validation"
test_dataset_path = "blocks/test"

In [ ]:
# Load label
with open(os.path.join(train_dataset_path, "labels.json"), "r") as f:
  labels_json = json.load(f)
for category_item in labels_json["categories"]:
  print(f"{category_item['id']}: {category_item['name']}")


# **Bagian 3: Visualisasi Dataset**

In [ ]:
# Import pustaka untuk visualisasi
import matplotlib.pyplot as plt
from matplotlib import patches, text, patheffects
from collections import defaultdict
import math

def draw_outline(obj):
  obj.set_path_effects([patheffects.Stroke(linewidth=4,  foreground='black'), patheffects.Normal()])

def draw_box(ax, bb):
  patch = ax.add_patch(patches.Rectangle((bb[0],bb[1]), bb[2], bb[3], fill=False, edgecolor='red', lw=2))
  draw_outline(patch)

def draw_text(ax, bb, txt, disp):
  text = ax.text(bb[0],(bb[1]-disp),txt,verticalalignment='top', color='white', fontsize=10, weight='bold')
  draw_outline(text)

def draw_bbox(ax, annotations_list, id_to_label, image_shape):
  for annotation in annotations_list:
    cat_id = annotation["category_id"]
    bbox = annotation["bbox"]
    draw_box(ax, bbox)
    draw_text(ax, bbox, id_to_label[cat_id], image_shape[0] * 0.05)

def visualize(dataset_folder, max_examples=None):
  with open(os.path.join(dataset_folder, "labels.json"), "r") as f:
    labels_json = json.load(f)
  images = labels_json["images"]
  cat_id_to_label = {item["id"]:item["name"] for item in labels_json["categories"]}
  image_annots = defaultdict(list)
  for annotation_obj in labels_json["annotations"]:
    image_id = annotation_obj["image_id"]
    image_annots[image_id].append(annotation_obj)

  if max_examples is None:
    max_examples = len(image_annots.items())
  n_rows = math.ceil(max_examples / 3)
  fig, axs = plt.subplots(n_rows, 3, figsize=(24, n_rows*8)) # 3 columns(2nd index), 8x8 for each image
  for ind, (image_id, annotations_list) in enumerate(list(image_annots.items())[:max_examples]):
    ax = axs[ind//3, ind%3]
    img = plt.imread(os.path.join(dataset_folder, "images", images[image_id]["file_name"]))
    ax.imshow(img)
    draw_bbox(ax, annotations_list, cat_id_to_label, img.shape)
  plt.show()

visualize(train_dataset_path, 5)

# **Bagian 4: Persiapan Dataset dan Training Model**

In [ ]:
# Fungsi untuk menghitung jumlah data dalam setiap kelas
def count_data_per_class(dataset_folder):
  with open(os.path.join(dataset_folder, "labels.json"), "r") as f:
    labels_json = json.load(f)
  class_counts = defaultdict(int)
  for annotation_obj in labels_json["annotations"]:
    class_counts[annotation_obj["category_id"]] += 1
  return class_counts

train_class_counts = count_data_per_class(train_dataset_path)
validation_class_counts = count_data_per_class(validation_dataset_path)
test_class_counts = count_data_per_class(test_dataset_path)

print("Train class counts:", train_class_counts)
print("Validation class counts:", validation_class_counts)
print("Test class counts:", test_class_counts)

In [ ]:
# Load dataset
train_data = object_detector.Dataset.from_coco_folder(train_dataset_path, cache_dir="/tmp/od_data/train")
validation_data = object_detector.Dataset.from_coco_folder(validation_dataset_path, cache_dir="/tmp/od_data/validation")
test_data = object_detector.Dataset.from_coco_folder(test_dataset_path, cache_dir="/tmp/od_data/test")
print("train_data size: ", train_data.size)
print("validation_data size: ", validation_data.size)
print("test_data size: ", test_data.size)

In [ ]:
# Spesifikasi model dan hyperparameters
spec = object_detector.SupportedModels.MOBILENET_MULTI_AVG

# Define hyperparameters
epochs = 30  # Ubah sesuai kebutuhan
steps_per_epoch = 12  # Ubah sesuai kebutuhan

hparams = object_detector.HParams(
    export_dir='exported_model',
    epochs=epochs,
    steps_per_epoch=steps_per_epoch
)

options = object_detector.ObjectDetectorOptions(
    supported_model=spec,
    hparams=hparams
)

In [ ]:
# Training model
model = object_detector.ObjectDetector.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options)

# **Bagian 5: Evaluasi Model**

In [ ]:
# Evaluasi model dengan dataset validasi
validation_loss, validation_coco_metrics = model.evaluate(validation_data, batch_size=4)
print(f"Validation loss: {validation_loss}")
print(f"Validation coco metrics: {validation_coco_metrics}")

In [ ]:
# Evaluasi model dengan dataset test
test_loss, test_coco_metrics = model.evaluate(test_data, batch_size=4)
print(f"Test loss: {test_loss}")
print(f"Test coco metrics: {test_coco_metrics}")

In [ ]:
# Ekspor model
model.export_model()
!ls exported_model
files.download('exported_model/model.tflite')

# **Bagian 6: Menghitung Metrik Tambahan**

In [ ]:
# Import pustaka untuk menghitung metrik
from sklearn.metrics import accuracy_score, f1_score, recall_score, confusion_matrix, ConfusionMatrixDisplay

In [ ]:
# Fungsi untuk mendapatkan nilai ground truth dan prediksi
def get_ground_truth_and_predictions(model, dataset):
    ground_truths = []
    predictions = []
    for data in dataset:
        image, labels = data
        pred = model.predict(image)
        for label in labels:
            ground_truths.append(label['category_id'])
        for detection in pred.detections:
            predictions.append(detection.label_id)
    return ground_truths, predictions


In [ ]:
# Mendapatkan ground truths dan predictions untuk dataset validation
validation_ground_truths, validation_predictions = get_ground_truth_and_predictions(model, validation_data)


In [ ]:
# Menghitung confusion matrix untuk dataset validation
validation_conf_matrix = confusion_matrix(validation_ground_truths, validation_predictions)
print(f"Validation Confusion Matrix: \n{validation_conf_matrix}")


In [ ]:
# Mendapatkan ground truths dan predictions untuk dataset test
test_ground_truths, test_predictions = get_ground_truth_and_predictions(model, test_data)


In [ ]:
# Menghitung confusion matrix untuk dataset test
test_conf_matrix = confusion_matrix(test_ground_truths, test_predictions)
print(f"Test Confusion Matrix: \n{test_conf_matrix}")

In [ ]:
# Fungsi untuk menampilkan confusion matrix
def plot_confusion_matrix(conf_matrix, class_names, title):
    disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=class_names)
    disp.plot(cmap=plt.cm.Blues)
    plt.title(title)
    plt.show()

In [ ]:
# Mendapatkan nama kelas
class_names = [item["name"] for item in labels_json["categories"]]


In [ ]:
# Menampilkan confusion matrix untuk dataset validation
plot_confusion_matrix(validation_conf_matrix, class_names, "Validation Confusion Matrix")


In [ ]:
# Menampilkan confusion matrix untuk dataset test
plot_confusion_matrix(test_conf_matrix, class_names, "Test Confusion Matrix")


# **Bagian 7: Menampilkan Grafik Training dan Evaluasi Model**

In [ ]:
# Fungsi untuk menampilkan grafik training dan evaluasi model
def plot_training_history(history):
    fig, axs = plt.subplots(2, 2, figsize=(12, 12))

    # Plot total_loss
    axs[0, 0].plot(history.history['loss'], label='Training Loss')
    axs[0, 0].plot(history.history['val_loss'], label='Validation Loss')
    axs[0, 0].set_title('Total Loss')
    axs[0, 0].set_xlabel('Epochs')
    axs[0, 0].set_ylabel('Loss')
    axs[0, 0].legend()

    # Plot cls_loss
    axs[0, 1].plot(history.history['cls_loss'], label='Training Class Loss')
    axs[0, 1].plot(history.history['val_cls_loss'], label='Validation Class Loss')
    axs[0, 1].set_title('Class Loss')
    axs[0, 1].set_xlabel('Epochs')
    axs[0, 1].set_ylabel('Loss')
    axs[0, 1].legend()

    # Plot box_loss
    axs[1, 0].plot(history.history['box_loss'], label='Training Box Loss')
    axs[1, 0].plot(history.history['val_box_loss'], label='Validation Box Loss')
    axs[1, 0].set_title('Box Loss')
    axs[1, 0].set_xlabel('Epochs')
    axs[1, 0].set_ylabel('Loss')
    axs[1, 0].legend()

    # Plot model_loss
    axs[1, 1].plot(history.history['model_loss'], label='Training Model Loss')
    axs[1, 1].plot(history.history['val_model_loss'], label='Validation Model Loss')
    axs[1, 1].set_title('Model Loss')
    axs[1, 1].set_xlabel('Epochs')
    axs[1, 1].set_ylabel('Loss')
    axs[1, 1].legend()

    plt.tight_layout()
    plt.show()

plot_training_history(history)

In [ ]:
# Fungsi untuk menampilkan grafik Average Precision (AP) dan Average Recall (AR)
def plot_evaluation_metrics(coco_metrics):
    fig, axs = plt.subplots(1, 2, figsize=(12, 6))

    # Plot Average Precision (AP)
    axs[0].bar(range(len(coco_metrics['AP'])), coco_metrics['AP'], label='Average Precision (AP)')
    axs[0].set_title('Average Precision (AP)')
    axs[0].set_xlabel('Classes')
    axs[0].set_ylabel('AP')
    axs[0].legend()

    # Plot Average Recall (AR)
    axs[1].bar(range(len(coco_metrics['AR'])), coco_metrics['AR'], label='Average Recall (AR)')
    axs[1].set_title('Average Recall (AR)')
    axs[1].set_xlabel('Classes')
    axs[1].set_ylabel('AR')
    axs[1].legend()

    plt.tight_layout()
    plt.show()

plot_evaluation_metrics(coco_metrics)

## Benchmarking
Below is a summary of our benchmarking results for the supported model architectures. These models were trained and evaluated on the same android figurines dataset as this notebook. When considering the model benchmarking results, there are a few important caveats to keep in mind:
* The android figurines dataset is a small and simple dataset with 62 training examples and 10 validation examples. Since the dataset is quite small, metrics may vary drastically due to variances in the training process. This dataset was provided for demo purposes and it is recommended to collect more data samples for better performing models.
* The float32 models were trained with the default HParams, and the QAT step for the int8 models was run with `QATHParams(learning_rate=0.1, batch_size=4, epochs=30, decay_rate=1)`.
* For your own dataset, you will likely need to tune values for both HParams and QATHParams in order to achieve the best results. See the [Hyperparameters](#hyperparameters) section above for more information on configuring training parameters.
* All latency numbers are benchmarked on the Pixel 6.


<table>
<thead>
<col>
<col>
<colgroup span="2"></colgroup>
<colgroup span="2"></colgroup>
<colgroup span="2"></colgroup>
<tr>
<th rowspan="2">Model architecture</th>
<th rowspan="2">Input Image Size</th>
<th colspan="2" scope="colgroup">Test AP</th>
<th colspan="2" scope="colgroup">CPU Latency</th>
<th colspan="2" scope="colgroup">Model Size</th>
</tr>
<tr>
<th>float32</th>
<th>QAT int8</th>
<th>float32</th>
<th>QAT int8</th>
<th>float32</th>
<th>QAT int8</th>
</tr>
</thead>
<tbody>
<tr>
<td>MobileNetV2</td>
<td>256x256</td>
<td>88.4%</td>
<td>73.5%</td>
<td>48ms</td>
<td>16ms</td>
<td>11MB</td>
<td>3.2MB</td>
</tr>
<tr>
<td>MobileNetV2 I320</td>
<td>320x320</td>
<td>89.1%</td>
<td>75.5%</td>
<td>75ms</td>
<td>33.38ms</td>
<td>10MB</td>
<td>3.3MB</td>
</tr>
<tr>
<td>MobileNet MultiHW AVG</td>
<td>256x256</td>
<td>88.5%</td>
<td>70.0%</td>
<td>56ms</td>
<td>19ms</td>
<td>13MB</td>
<td>3.6MB</td>
</tr>
<tr>
<td>MobileNet MultiHW AVG I384</td>
<td>384x384</td>
<td>92.7%</td>
<td>73.4%</td>
<td>238ms</td>
<td>41ms</td>
<td>13MB</td>
<td>3.6MB</td>
</tr>

</tbody>
</table>

